# Sequences and Loops in Parsl

In this notebook, we will be walking through examples of implementation of Parsl in different kinds of sequences and loops. Specifically, this notebook contains the implementations of the following Sequences:

1. For loops, in a sequence.
2. Nested Loops
3. Synchronization
4. Exclusive Routing (Type 1): Step by step evaluation
5. Exclusive Routing (Type 2): Continuous evaluation
6. Exclusive Routing (Type 3): Filtering and Aggregration 

### Importing Libraries and Configuration

We'll be using the htex configuration for Parsl. Read more [here.]( https://github.com/Parsl/parsl/blob/master/parsl/configs/htex_local.py)

In [1]:
import numpy as np

import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config

parsl.load(config)

### For loops in Sequence

Since sequences are executed chronologically, their steps can't be executed in parallel. For example, if f(x) is followed by g(x) which is followed by h(x), then we can't execute them in parallel since their inputs depend on the previous functions results. But we can execute multiple sequences at the same time.

For example, Let x be an input that has to be processed by f(x), g(x) and h(x). Then we can execute parallel sequences for 100 values of x as shown below. Note: f(x), g(x) and h(x) are all randomly selected functions can be changed as per the need of the experiment


<img src="./images/parallel_sequences.png"
     style="float: left; size: 15px;" />

In [2]:
## Building the Python Apps for f(x), g(x) and h(x)

@python_app
def f(x):
    return (x**2 - 4)

@python_app
def g(x):
    import numpy as np
    return (np.sin(x) + np.cos(x))

@python_app
def h(x):
    import numpy as np
    return (np.exp(x))

In [3]:
x = np.linspace(1,100,100) # This is out input list of first 100 natural number

In [4]:
# We evaluate 3 For loops on parallel

out_f = []
out_g = []
out_h = []

for element in x:
    out_f.append(f(element))

for element2 in out_f:
    out_g.append(g(element2))

for element3 in out_g:
    out_h.append(h(element3))
    
out_h = [i.result() for i in out_h]

An important thing to note in the above example is that we didn't have to evaluate the results of f(x) or g(x) before h(x) using the .result() function. We simply parsed the AppFuture objects into the next python app without evaluation. Thus, it reduces the number of separate evaluations conducted and pushes everything to be conducted in parallel at the end.

In [5]:
# Checking the length of the results

print(len(out_f))
print(len(out_g))
print(len(out_h))

100
100
100


### Nested Loops

A nested loop is a loop within a loop. Here, one loop gives us a result which becomes the input of another loop. For example, The function 'natural numbers' generates a list of the 20 natural numbers after the given input 'x' and 'square roots' generates a list of a given number's square root

<img src="./images/nested_looping.png"
     style="float: left; size: 15px;" />

In [6]:
x = np.linspace(1,50,50)

# Building the python apps for initiating the intermediate lists and then evaluating them

@python_app
def natural_numbers(x):
    import numpy as np
    return np.linspace(x+1, (19+x),20)

@python_app
def square_roots(x):
    return [+x**0.5,-x**0.5]

In [7]:
# Using for loops to generate the intermediate lists and using the output to evaluate the second function

final_array = []

for element in x:
    output_list = natural_numbers(element)

    for element2 in output_list.result():
        final_array.append(square_roots(element2))
        
final_array = [i.result() for i in final_array]

In [8]:
print(len(final_array))

1000


### Synchronization

Synchronization is the merge of existing parallel threads to compute one final result. For example, 100 different natural numbers are first squared and then their average is evaluated

<img src="./images/synchronization.png"
     style="float: left; size: 10px;" />

In [9]:
# Building a python funcion to evaluate the results and then take the final mean

@python_app
def square(x):
    return x**2

@python_app
def average(inputs=[]):
    return sum(inputs)/len(inputs)
    
output_list = []
for i in range(1,5):
    output_list.append(square(i))

mean = average(inputs=output_list).result()

In [10]:
print(mean)

7.5


### Exclusive Routing 1

Exclusive Routing is the idea of parsing a result to a parallel branch if it meets a given condition

<img src="./images/exclusive_routing1.png"
     style="float: left; size: 15px;" />

In [11]:
inputs = np.linspace(1,100,100)

# Using a check function to route the input to either of the python apps

def check(x):
    if x%3 == 0 and x%5 == 0:
        return True
    else:
        return False
    
# Python app for taking the square root of the function
@python_app
def square_root(x):
    return x**0.5

# Python app for taking the square of the function
@python_app
def squared(x):
    return x**2

final_list = []

for x in inputs:
    if check(x):
        final_list.append(square_root(x))
    else:
        final_list.append(squared(x))

In [12]:
# Evaluating final results
final_list = [i.result() for i in final_list]

In [13]:
print(len(final_list))

100


### Exclusive Routing 2 ( Continuous Evaluation )

This type of exclusive Routing is the idea of parsing a result to given function repeatedly until it meets a condition. For example, a function takes the square root of a number until it becomes less than 2. 

<img src="./images/exclusive_routing_2.png"
     style="float: left; size: 15px;" />

Let's consider a simple version of continuous evaluation (as shown in the workflow above):

In [14]:
# Python app for taking the square root of the function
@python_app
def square_root(x):
    return x**0.5

x = 1000

x = square_root(x)

while x.result() > 2:
    x = square_root(x)

In [15]:
x.result()

1.539926526059492

Let us consider a restricted but more efficient and parallel version of continuous evaluation which compares the inputs as part of the parallel thread rather than the main thread:

In [16]:
# Python app for taking the square root of the function
@python_app
def square_root(x):
    return x**0.5

# Python app for comparing the square root of the function
@python_app
def compare_square_root(x):
    if x > 2:
        return True
    else:
        False

def convergence(x):
    
    x = square_root(x)
    
    for i in range(100):
        if compare_square_root(x):
            x = square_root(x)
        else:
            break
    
    if compare_square_root(x):
        return True, x  ## Converged over 100 iterations
    else:
        return False, x ## Didn't converge over 100 iterations

In [17]:
inputs = np.linspace(101,200,100)
outputs = []

for i in inputs:
    state, output = convergence(i)
    outputs.append(output)

In [18]:
outputs = [i.result() for i in outputs]

### Exclusive Routing 3 ( Filtering and Aggregation)

This type of exclusive Routing is the idea of parsing a result to given function and evaluating if it would be part of the aggregate function (synchronization at the end). For example, a result(x) evaluates the sin(x) + cos(x). In the end, we want the mean of all the positive outputs.

<img src="./images/exclusive_routing_3.png"
     style="float: left; size: 15px;" />

In [19]:
# Python app for evaluating the sum of sine and cosine of x.
@python_app
def result(x):
    import numpy as np
    return np.sin(x) + np.cos(x)

# Python app for comparing the input
@python_app
def compare(x):
    if x > 0:
        return True
    else:
        return False

# Python app for computing the confidence intervals of the inputs list.
@python_app
def percentiles(inputs = []):
    import numpy as np
    return np.percentile(inputs,25), np.percentile(inputs,75)

In [20]:
inputs = np.linspace(0,2*np.pi,100)
filtered = []

for element in inputs:
    output = result(element)
    if compare(output):
        filtered.append(output)
        
final_result = percentiles(inputs=filtered).result()

In [21]:
print(final_result)

(-0.9797614795410088, 0.9999999999999998)
